<a href="https://colab.research.google.com/github/ejrtks1020/Pytorch-basic/blob/main/PyTorch_CNN_MNIST_%EB%B6%84%EB%A5%98_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax

In [22]:
import torch
import torch.nn as nn


In [2]:
# 배치 크기 x 채널 x 높이(height) x 너비(width)크기의 임의의 텐서를 선언
inputs = torch.Tensor(1, 1, 28, 28)
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


# 합성곱층과 풀링 선언하기

1채널 짜리를 입력받아서 32채널을 출력, 커널 사이즈는 3이고 패딩은 1

In [5]:
conv1 = nn.Conv2d(1, 32 ,kernel_size = 3, stride = 1, padding = 1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


두번째 합성곱 층, 32 채널을 입력받아서 64채널을 출력, 커널사이즈는 3이고 패딩은 1

In [6]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding = 1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


맥스풀링 구현, 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 모두 해당값으로 지정됨

In [7]:
pool = nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


# 구현체를 연결하여 모델 만들기

입력을 첫번째 합성곱층을 통과시키고 텐서의 크기를 출력

In [10]:
out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


32채널, 28너비 28높이의 텐서

In [11]:
# 맥스 풀링 통과
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


맥스풀링을 통과한 텐서를 두번째 합성곱층에 입력

In [12]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [13]:
# 맥스 풀링 통과
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [14]:
# 텐서의 0, 1, 2, 3 차원의 값을 출력
out.size(0), out.size(1), out.size(2), out.size(3)

(1, 64, 7, 7)

In [15]:
# .view()를 이용해 텐서를 펼친다.
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


배치 차원을 제외하고 모두 하나의 차원으로 통합되었다. 이것을 전결합층(Fully-Connected layer)으로 통과시킨다. 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환한다.

In [16]:
fc = nn.Linear(3136, 10) # input_dim = 3136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


# CNN으로 MNIST 분류하기

In [17]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [18]:
# 만약 GPU를 사용 가능하다면 device값이 cuda가 되고, 아니라면 cpu가 된다
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [19]:
# 학습에 사용할 하이퍼 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

데이터로더를 사용하여 데이터를 다루기 위해서 데이터셋 정의

In [20]:
mnist_train = dsets.MNIST(root = 'MNIST_data/', #다운로드 경로지정
                          train =True,  # 학습데이터로 다운로드
                          transform = transforms.ToTensor(), #텐서로 변환
                          download =True)

mnist_test = dsets.MNIST(root = 'MNIST_data/', #다운로드 경로지정
                          train =False, #테스트 데이터로 다운로드
                          transform = transforms.ToTensor(), #텐서로 변환
                          download =True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [21]:
#데이터로더를 사용하여 배치 크기를 지정한다
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [26]:
# 클래스로 모델 설계

class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # 첫번째 층
    # Imgin Shape = (? , 28, 28, 1)
    # Conv  =       (? , 28, 28, 32)
    # Pool  =       (? , 14, 14, 32)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride = 1, padding =1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
    )

    # 두번째 층
    # ImgIn shape = (? , 14, 14, 32)
    # Conv  =       (? , 14, 14, 64)
    # Pool  =       (? , 7, 7, 64)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride = 2)
    )

    # 전결합층 7x7x64 -> 10 outputs
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias = True)
    
    # 전결합층 한정으로 가중치 초기화
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1) # 전 결합층을 위해서 Flatten
    out = self.fc(out)

    return out

In [27]:
# CNN 모델 정의
model = CNN().to(device)

In [28]:
# 비용함수와 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용함수에 소프트 맥스 함수 포함
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [30]:
# 총 배치의 수 출력
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


총 배치의 수는 600이다. 배치 크기를 100으로 했으므로 훈련데이터는 총 60000개란 의미

In [31]:
# 모델 훈련

for epoch in range(training_epochs):
  avg_cost = 0

  for X, y in data_loader: # 미니 배치 단위로 꺼내온다, X는 미니 배치, y는 레이블
  # 이미지는 이미 28 x 28
    X = X.to(device)
    y = y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

[Epoch:    1] cost = 0.225644171
[Epoch:    2] cost = 0.0629913285
[Epoch:    3] cost = 0.0461514555
[Epoch:    4] cost = 0.0374220833
[Epoch:    5] cost = 0.031358961
[Epoch:    6] cost = 0.0260042455
[Epoch:    7] cost = 0.021739101
[Epoch:    8] cost = 0.0181058664
[Epoch:    9] cost = 0.0159733519
[Epoch:   10] cost = 0.0131852636
[Epoch:   11] cost = 0.0100417919
[Epoch:   12] cost = 0.00904354826
[Epoch:   13] cost = 0.0091658961
[Epoch:   14] cost = 0.00647167582
[Epoch:   15] cost = 0.00663798302


In [34]:
# 테스트
# 학습을 진행하지 않으므로 torch.no_grad()
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  y_test = mnist_test.test_labels.to(device)
  prediction = model(X_test)
  print(prediction.shape)
  correct_prediction = torch.argmax(prediction, 1) == y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


torch.Size([10000, 10])
Accuracy: 0.9865000247955322
